In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

for doc in documents:
    
    bag = []
   
    pattern_words = doc[0]
    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
   
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

# model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

29 documents
11 classes ['Further Guidance', 'goodbye', 'greeting', 'hod_BSH', 'hod_CHEM', 'hod_CIVIL', 'hod_CSE', 'hod_ECE', 'options', 'rankings', 'thanks']
61 unique lemmatized words ['&', "'s", ',', 'about', 'anyone', 'are', 'awesome', 'b', 'be', 'bye', 'can', 'chatting', 'chem', 'civil', 'college', 'could', 'cse', 'current', 'day', 'do', 'ece', 'for', 'gmrit', 'good', 'goodbye', 'h', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hod', 'hola', 'how', 'i', 'is', 'know', 'later', 'me', 'more', 'next', 'nice', 'of', 'offered', 'provide', 'ranking', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your']
Training data created
Epoch 1/200


<ipython-input-5-38fca3719504>:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


6/6 [==============================] - 1s 2ms/step - loss: 2.3630 - accuracy: 0.1062
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 2.3084 - accuracy: 0.1980
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 2.2471 - accuracy: 0.2722
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 2.1715 - accuracy: 0.2598
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 2.0148 - accuracy: 0.3500
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8451 - accuracy: 0.4283
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6514 - accuracy: 0.4036
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5595 - accuracy: 0.5842
Epoch 9/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8244 - accuracy: 0.5364
Epoch 10/200
6/6 [==============================] - 0s 4ms/step - loss: 1.6085 - accuracy: 0.4446
Epoch 11/200
6/6 [=======================